In [15]:
from random import random

from itertools import islice

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql.json import JSONB
from  sqlalchemy.sql.expression import func

from poultry import readline_dir
from poultry import tweet

In [2]:
engine = create_engine('postgresql://dm303@localhost/sample')

In [3]:
Session = sessionmaker(bind=engine)

In [4]:
session = Session()

In [5]:
Base = declarative_base()

In [6]:
class Tweet(Base):
    __tablename__ = 'tweets'
    
    id = Column(Integer, primary_key=True)
    
    raw_data = Column(JSONB)

In [7]:
Base.metadata.create_all(engine)

In [8]:
%%time

for t in readline_dir('/import/dima-scratch/flock/data/brexit.sample.122K/'):
    session.add(Tweet(raw_data=t.parsed))

CPU times: user 2min 16s, sys: 5.84 s, total: 2min 22s
Wall time: 2min 21s


In [10]:
%%time
session.commit()

CPU times: user 1min 51s, sys: 6.62 s, total: 1min 58s
Wall time: 3min 37s


In [14]:
session.query(Tweet).count()

122567L

In [11]:
session.query(Tweet).filter(Tweet.raw_data.contains({'lang': 'en'})).count()

81456L

In [12]:
session.query(Tweet).filter(Tweet.raw_data.contains({'lang': 'it'})).count()

3601L

In [13]:
session.query(Tweet).filter(Tweet.raw_data.contains({'lang': 'de'})).count()

2641L

In [29]:
%%time
# Random selection of tweets.
[t.raw_data['id'] for t in session.query(Tweet).order_by(func.random()).limit(10)]

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 194 ms


[746541958848516097,
 746283104101404672,
 746405254426730500,
 746779962909286400,
 746379909237354496,
 746183770915512320,
 746575848405274624,
 746224773445758976,
 746708424231165952,
 746510857866194944]